<a href="https://colab.research.google.com/github/GiX7000/10-machine-learning-algorithms-from-scratch/blob/main/03_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementation of Logistic Regression classifier

Instead of predicting just a continuous value, classification tasks most of times require a non-linear approach, in which we try to predict probabilities. A brief introduction to logistic  regression can be found [here](https://www.youtube.com/watch?v=72AHKztZN44), while for deeper exploration, see this [lecture](https://www.youtube.com/watch?v=4u81xU7BIOc). Here, we will see a quick and dirty implementation of a logistic regression classifier.

In [1]:
# import libraries
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
import matplotlib.pyplot as plt

## 1. Download [breast cancer dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html) and visualize it.

In [2]:
# download the dataset
bc = datasets.load_breast_cancer()

# retrive X, y
X, y = bc.data, bc.target

# create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# let's see some things about the data
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
print(X_train.dtype, X_test.dtype)
print(X_train[5])
print(y_train[5])

(455, 30) (455,) (114, 30) (114,)
float64 float64
[1.625e+01 1.951e+01 1.098e+02 8.158e+02 1.026e-01 1.893e-01 2.236e-01
 9.194e-02 2.151e-01 6.578e-02 3.147e-01 9.857e-01 3.070e+00 3.312e+01
 9.197e-03 5.470e-02 8.079e-02 2.215e-02 2.773e-02 6.355e-03 1.739e+01
 2.305e+01 1.221e+02 9.397e+02 1.377e-01 4.462e-01 5.897e-01 1.775e-01
 3.318e-01 9.136e-02]
0


## 2. Create the Linear Regression model.

To take the probabilities of a binary classification task, we will use the [sigmoid function](https://www.youtube.com/watch?v=TPqr8t919YM&t=135s). Instead of MSE, the loss function in this case is the [binary cross-entropy loss](https://towardsdatascience.com/understanding-binary-cross-entropy-log-loss-a-visual-explanation-a3ac6025181a), while [gradient descent](https://www.geeksforgeeks.org/gradient-descent-algorithm-and-its-variants/) also helps us to minimize it. We use them as you can see in the below cells.

In [3]:
# define the sigmoid function
def sigmoid(x):
  # clip to avoid overflow in exp large values
  x = np.clip(x, -500, 500)
  return 1 / (1 + np.exp(-x))

In [4]:
# create the linear regression classifier as a class
class my_LogisticRegression:

  # init function to define what elements(initialized or not) we are going to use inside this class
  def __init__(self, lr=0.001, n_iters=1000):
    self.lr = lr
    self.n_iters = n_iters
    # set weights and bias to None for now
    self.weights = None
    self.bias = None

  # fit function for training
  def fit(self, X, y):
    # get the shape of the X input
    n_samples, n_features = X.shape
    # initialize weights and bias to 0 in the appropriate shape. Remember: y = X*w + b
    self.weights = np.zeros(n_features)
    self.bias = 0

    for _ in range(self.n_iters):
      # forward pass to calculate the y logits
      y_logits = np.dot(X, self.weights) + self.bias
      # take the probabilities of y logits(just add an activation function to our linear predictions)
      y_pred = sigmoid(y_logits)

      # calculate gradients of w and b
      dw = (1/n_samples) * np.dot(X.T, (y_pred - y))
      db = (1/n_samples) * np.sum(y_pred - y)

      # update weights and bias
      self.weights -= self.lr * dw
      self.bias -= self.lr * db


  # predict function for inference
  def predict(self, X):
    # forward pass: just take the probabbilities of y logits
    y_pred = sigmoid(np.dot(X, self.weights) + self.bias)
    # prediction=0 if y_pred<0.5, else prediction=1
    y_pred_cls = [1 if i > 0.5 else 0 for i in y_pred]
    return y_pred_cls

## 3. Train, predict and evaluate the model.

In [5]:
# create ana instance of our linear classifier
clf_log_reg = my_LogisticRegression()

# train the model
clf_log_reg.fit(X_train, y_train)

In [6]:
# predict on test set
predictions = clf_log_reg.predict(X_test)

# Print the predictions and the actual labels
print("Predictions:", np.array(predictions))
print("Actual Labels:", y_test)

Predictions: [1 1 1 1 1 1 0 1 0 0 0 0 1 0 1 0 1 1 1 0 1 0 0 0 0 0 0 1 1 1 1 1 0 1 1 1 1
 0 0 1 0 1 0 0 1 1 1 0 0 0 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 0 0 1 1 0 1 0 1 1
 0 1 1 0 1 0 1 1 0 1 0 0 0 0 0 1 1 0 1 0 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0
 0 0 0]
Actual Labels: [1 1 1 1 1 1 0 1 0 0 0 1 1 1 1 0 1 1 1 0 1 0 0 0 0 1 0 1 1 1 1 1 0 1 1 1 1
 0 0 1 0 1 0 1 1 1 1 0 0 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 0 0 1 1 0 1 0 1 0
 1 1 1 0 1 0 1 1 1 0 0 0 0 0 1 0 1 0 1 0 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0
 1 0 0]


Let's use how good our model is.

In [7]:
# accuracy function
def accuracy(y_true, y_pred):
  return np.sum(y_true == y_pred) / len(y_true)

acc = accuracy(y_test, predictions)
print(acc)

0.8947368421052632


It could be better. Let's try for a bigger  learning rate.

In [8]:
# create ana instance of our linear classifier
clf_log_reg_2 = my_LogisticRegression(lr=0.01)

# train the model
clf_log_reg_2.fit(X_train, y_train)

In [9]:
# predict on test set
predictions = clf_log_reg_2.predict(X_test)

# let's see now the new mse and r2
acc_2 = accuracy(y_test, predictions)
print(acc_2)

0.9210526315789473


For bigger learbing rate, we got better accuracy.

## 4. Compare with LinearRRegression classifier from scikit-learn library.

Let's see now, what results a [LinearRegression classifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) from scikit-learn library gives us.

In [10]:
# let's compare now with the accuracy that sklearn gives us
from sklearn.linear_model import LogisticRegression

clf_sklearn = LogisticRegression(max_iter=100)
clf_sklearn.fit(X_train, y_train)
sklearn_predictions = clf_sklearn.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
# let's check the accuracy now
acc_sklearn = accuracy(y_test, sklearn_predictions)
print(acc_sklearn)

0.9385964912280702


In this case, we get better results from our above custom classifier.